In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
import keras_tuner as kt

In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [4]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [5]:
img_train.dtype

dtype('float32')

Can make hypermodel in two ways:
* Use model builder function.
* Subclass HyperModel class

In [6]:
kt.applications.efficientnet.HyperEfficientNet

keras_tuner.applications.efficientnet.HyperEfficientNet

In [7]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28,28)))
    
    # Choose optimal number between 32 and 512, with +-32
    hp_units = hp.Int('units',min_value=32,max_value=512,step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(10))
    # Choose learning rate from 1e-2, 1e-3 or 1e-4.
    hp_lr = hp.Choice('learning_rate', values=[1e-2,1e-3,1e-4])
    model.compile(
        optimizer=keras.optimizers.Adam(hp_lr),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'],
    )
    return model

In [8]:
tuner = kt.Hyperband(model_builder,
                    objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt',
                    )

2022-04-05 09:30:52.227712: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [9]:
stop_early = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',patience=5)

In [10]:
import numpy as np
1 + np.log(10,)/np.log(3)

3.095903274289385

In [11]:
tuner.search(
    img_train, 
    label_train,
    epochs=50,
    validation_split=.2,
    callbacks=[stop_early],
)

Trial 30 Complete [00h 00m 37s]
val_accuracy: 0.856249988079071

Best val_accuracy So Far: 0.8947499990463257
Total elapsed time: 00h 08m 04s


In [12]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [13]:
best_hps.get_config()

{'space': [{'class_name': 'Int',
   'config': {'name': 'units',
    'default': None,
    'conditions': [],
    'min_value': 32,
    'max_value': 512,
    'step': 32,
    'sampling': None}},
  {'class_name': 'Choice',
   'config': {'name': 'learning_rate',
    'default': 0.01,
    'conditions': [],
    'values': [0.01, 0.001, 0.0001],
    'ordered': True}}],
 'values': {'units': 480,
  'learning_rate': 0.001,
  'tuner/epochs': 10,
  'tuner/initial_epoch': 0,
  'tuner/bracket': 0,
  'tuner/round': 0}}

In [14]:
print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")



The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 480 and the optimal learning rate for the optimizer
is 0.001.



In [15]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.4908 - accuracy: 0.8259 - val_loss: 0.3922 - val_accuracy: 0.8593
Epoch 2/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3672 - accuracy: 0.8664 - val_loss: 0.3747 - val_accuracy: 0.8683
Epoch 3/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3330 - accuracy: 0.8765 - val_loss: 0.3500 - val_accuracy: 0.8726
Epoch 4/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3057 - accuracy: 0.8865 - val_loss: 0.3248 - val_accuracy: 0.8817
Epoch 5/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2876 - accuracy: 0.8928 - val_loss: 0.3120 - val_accuracy: 0.8879
Epoch 6/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2693 - accuracy: 0.8991 - val_loss: 0.3518 - val_accuracy: 0.8749
Epoch 7/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2565 - accuracy: 0.9033 - val_loss: 0.3765 - val_accuracy:

In [16]:
best_epoch

24

In [17]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=10, validation_split=0.2)

Epoch 1/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.4939 - accuracy: 0.8263 - val_loss: 0.4361 - val_accuracy: 0.8357
Epoch 2/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3699 - accuracy: 0.8643 - val_loss: 0.3627 - val_accuracy: 0.8671
Epoch 3/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3304 - accuracy: 0.8776 - val_loss: 0.3270 - val_accuracy: 0.8799
Epoch 4/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3036 - accuracy: 0.8868 - val_loss: 0.3460 - val_accuracy: 0.8746
Epoch 5/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2860 - accuracy: 0.8942 - val_loss: 0.3263 - val_accuracy: 0.8852
Epoch 6/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2716 - accuracy: 0.8974 - val_loss: 0.3182 - val_accuracy: 0.8884
Epoch 7/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2567 - accuracy: 0.9031 - val_loss: 0.3306 - val_accuracy:

In [18]:
tf.saved_model.save(hypermodel,'saved-model')

2022-04-05 09:43:21.029862: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [19]:
loaded = tf.saved_model.load('saved-model/')

In [20]:
loaded(np.random.uniform(size=(1,28,28)).astype('float32'))

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[ -4.023811 , -12.484087 ,   1.5164313,  -5.167639 ,  -5.073719 ,
        -10.617945 ,  -3.5240197, -11.719722 ,  10.838055 , -13.377506 ]],
      dtype=float32)>